In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [18]:
from langchain.schema.runnable import Runnable
from langchain_core.messages.ai import AIMessage

# Experiment with a Runnable that can convert a (list of) AIMessage objects to a list of strings
class AIMessageToString(Runnable):
    def invoke(self, input_messages, config):
        print(f"config: {config}")
        # Check if input_messages is a list
        if isinstance(input_messages, list):
            # Initialize an empty list to store content strings
            content_strings = []
            # Loop over each message in the list
            for message in input_messages:
                # Check if the message is a AIMessages instance
                if isinstance(message, AIMessage):
                    # Append the content string to the list
                    content_strings.append(message.content)
                else:
                    raise TypeError("Expected each item in the list to be a AIMessage object.")
            # Return the list of content strings
            return content_strings
        else:
            # Check if the message is just an AIMessages instance
            if isinstance(input_messages, AIMessage):
                # Return the content string
                return input_messages.content
            else:
                raise TypeError("Expected input_message to be a AIMessage object.")
        

In [19]:
ai_parser = AIMessageToString()

chain = llm | ai_parser

chain.invoke("How will the weather be in munich today?")

config: {'tags': [], 'metadata': {}, 'callbacks': <langchain_core.callbacks.manager.CallbackManager object at 0x10f518650>, 'recursion_limit': 25, 'configurable': {}}


"I'm sorry, I cannot provide real-time weather information. I recommend checking a reliable weather website or app for the most up-to-date forecast for Munich."

In [21]:
from langchain_core.tools import tool

@tool
def fake_weather_api(city: str) -> str:
    """Check the weather in a specified city."""
    return "Sunny, 22°C"

@tool
def outdoor_seating_availability(city: str) -> str:
    """Check if outdoor seating is available at a specified restaurant in a given city."""
    return "Outdoor seating is available."


tools = [fake_weather_api, outdoor_seating_availability]

It also possible to define tools using pydantic, see the comments below.

In [ ]:
# from langchain_core.pydantic_v1 import BaseModel, Field

# class WeatherCheck(BaseModel):
#     """Check the weather in a specified city."""

#     city: str = Field(..., description="Name of the city to check the weather for")


# class OutdoorSeatingCheck(BaseModel):
#     """Check if outdoor seating is available at a specified restaurant in a given city."""

#     city: str = Field(..., description="Name of the city where the restaurant is located")


# tools = [WeatherCheck, OutdoorSeatingCheck]

In [22]:
llm_with_tools = llm.bind_tools(tools)

In [26]:
llm_with_tools.invoke("How will the weather be in munich today?")


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_JG0Oq1OdGSW5EkcolQRNIrOK', 'function': {'arguments': '{"city":"Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 90, 'total_tokens': 107, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d0188d54-26d6-4a0b-b730-a5a912cf046d-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'Munich'}, 'id': 'call_JG0Oq1OdGSW5EkcolQRNIrOK', 'type': 'tool_call'}], usage_metadata={'input_tokens': 90, 'output_tokens': 17, 'total_tokens': 107})

In [27]:
result = llm_with_tools.invoke("How will the weather be in munich today? I would like to eat outside if possible")
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_OSWkntA9FYt9hQKrTaDlGA8I', 'function': {'arguments': '{"city": "Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_XjX5SJ17jqLrrgzcRoy89asF', 'function': {'arguments': '{"city": "Munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 98, 'total_tokens': 150, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-cbbc2503-cd4f-4009-8b77-b7397ceab51c-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'Munich'}, 'id': 'call_OSWkntA9FYt9hQKrTaDlGA8I', 'type': 'tool_call'}, {'name': 'outdoor_seating_availability', 'args': {'city': 'Munich'}, 'id': 'call_XjX5SJ17jqLrrgzcRoy89asF', 'type': 'tool_call'}], usage_metadata={'input_tokens': 98, 'output_tokens': 52,

In [28]:
result.tool_calls

[{'name': 'fake_weather_api',
  'args': {'city': 'Munich'},
  'id': 'call_OSWkntA9FYt9hQKrTaDlGA8I',
  'type': 'tool_call'},
 {'name': 'outdoor_seating_availability',
  'args': {'city': 'Munich'},
  'id': 'call_XjX5SJ17jqLrrgzcRoy89asF',
  'type': 'tool_call'}]

In [29]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [HumanMessage("How will the weather be in munich today? I would like to eat outside if possible")]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [30]:
messages

[HumanMessage(content='How will the weather be in munich today? I would like to eat outside if possible', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_16Q0jRaqZMDeAqTMRAgCQUuO', 'function': {'arguments': '{"city": "munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_AJhKbNufgobpJ3J5TvbO2B2D', 'function': {'arguments': '{"city": "munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 98, 'total_tokens': 148, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-895f63da-c167-4177-a3e9-d5bd503b0c7d-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'munich'}, 'id': 'call_16Q0jRaqZMDeAqTMRAgCQUuO', 'type': 'tool_call'}, {'name': 'outdoor_seating_availabil

In [31]:
tool_mapping = {"fake_weather_api": fake_weather_api, "outdoor_seating_availability": outdoor_seating_availability}

In [32]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    # NOTE: each tool implements the Runnable interface!
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [33]:
messages

[HumanMessage(content='How will the weather be in munich today? I would like to eat outside if possible', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_16Q0jRaqZMDeAqTMRAgCQUuO', 'function': {'arguments': '{"city": "munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_AJhKbNufgobpJ3J5TvbO2B2D', 'function': {'arguments': '{"city": "munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 98, 'total_tokens': 148, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-895f63da-c167-4177-a3e9-d5bd503b0c7d-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'munich'}, 'id': 'call_16Q0jRaqZMDeAqTMRAgCQUuO', 'type': 'tool_call'}, {'name': 'outdoor_seating_availabil

In [35]:
ai_parser = AIMessageToString()

chain = llm_with_tools | ai_parser

chain.invoke(messages)

config: {'tags': [], 'metadata': {}, 'callbacks': <langchain_core.callbacks.manager.CallbackManager object at 0x10ee6cef0>, 'recursion_limit': 25, 'configurable': {}}


'The weather in Munich today is sunny with a temperature of 22°C. Outdoor seating is available, so you can enjoy your meal outside.'

### Real third party API Call

In [ ]:
from langchain_core.tools import tool
import httpx

@tool
def fake_weather_api(city: str) -> str:
    """Check the weather in a specified city from a FastAPI endpoint on localhost:8000."""
    # Make an HTTP GET request to the FastAPI endpoint
    response = httpx.get(f"http://localhost:8000/weather?city={city}")

    if response.status_code == 200:
        return response.json().get("weather", "Weather information not available")
    else:
        return "Failed to get weather information"


@tool
def outdoor_seating_availability(city: str) -> str:
    """Check if outdoor seating is available in a specified city from a FastAPI endpoint on localhost:8000."""
    # Make an HTTP GET request to the FastAPI endpoint
    response = httpx.get(f"http://localhost:8000/outdoor-seating?city={city}")

    if response.status_code == 200:
        return response.json().get("outdoor_seating", "Outdoor seating information not available")
    else:
        return "Failed to get outdoor seating information"


api_tools = [fake_weather_api, outdoor_seating_availability]
tool_mapping = {"fake_weather_api": fake_weather_api, "outdoor_seating_availability": outdoor_seating_availability}

In [ ]:
from langchain_openai import ChatOpenAI

def interact_with_llm_and_tools(human_message: str):

    llm = ChatOpenAI()
    llm_with_tools_new = llm.bind_tools(api_tools)

    messages = [human_message]

    llm_output = llm_with_tools_new.invoke(messages)
    messages.append(llm_output)

    for tool_call in llm_output.tool_calls:
        tool_name = tool_call["name"].lower()
        tool = tool_mapping.get(tool_name)

        if tool:
            tool_output = tool.invoke(tool_call["args"])
            messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

    final_response = llm_with_tools_new.invoke(messages)
    return final_response

In [ ]:
interact_with_llm_and_tools("How will the weather be in sunnytown today? I would like to eat outside if possible")

In [ ]:
interact_with_llm_and_tools("How will the weather be in rainytown today? I would like to eat outside if possible")

In [ ]:
interact_with_llm_and_tools("How will the weather be in munich today? I would like to eat outside if possible")